In [273]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
import pandas as pd
import schedule
import time

In [275]:
# Define URLs for each zodiac sign from chosen website 
Aries_url = 'https://www.californiapsychics.com/horoscope/Aries-daily-horoscope/'
Taurus_url = 'https://www.californiapsychics.com/horoscope/Taurus-daily-horoscope/'
Gemini_url = 'https://www.californiapsychics.com/horoscope/gemini-daily-horoscope/'
Cancer_url = 'https://www.californiapsychics.com/horoscope/Cancer-daily-horoscope/'
Leo_url = 'https://www.californiapsychics.com/horoscope/Leo-daily-horoscope/'
Virgo_url = 'https://www.californiapsychics.com/horoscope/Virgo-daily-horoscope/'
Libra_url = 'https://www.californiapsychics.com/horoscope/Libra-daily-horoscope/'
Scorpio_url = 'https://www.californiapsychics.com/horoscope/Scorpio-daily-horoscope/'
Sagittarius_url = 'https://www.californiapsychics.com/horoscope/Sagittarius-daily-horoscope/'
Capricorn_url = 'https://www.californiapsychics.com/horoscope/Capricorn-daily-horoscope/'
Aquarius_url = 'https://www.californiapsychics.com/horoscope/Aquarius-daily-horoscope/'
Pisces_url = 'https://www.californiapsychics.com/horoscope/Pisces-daily-horoscope/'

In [277]:
#define function to get the daily horoscope from the website based on which horoscope is selected
def horoscope_strip(Sign):

    urls = {
        'Aries': Aries_url,
        'Taurus': Taurus_url,
        'Gemini': Gemini_url,
        'Cancer': Cancer_url,
        'Leo': Leo_url,
        'Virgo': Virgo_url,
        'Libra': Libra_url,
        'Scorpio': Scorpio_url,
        'Sagittarius': Sagittarius_url,
        'Capricorn': Capricorn_url,
        'Aquarius': Aquarius_url,
        'Pisces': Pisces_url
    }

#minor error handling... shouldnt be an issue but just in case there is a typo somewhere 
    if Sign not in urls:
        print("Invalid zodiac sign.")
        return

#get info from website given which (zodiac)sign we are calling 
    response = requests.get(urls[Sign])


    if response.status_code == 200:
        # Parse the webpage content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find paragraphs in the HTML
        paragraphs = soup.find_all('p')  

        # Extract and return the first paragraph's text, if it exists
        if paragraphs:
            horoscope = paragraphs[0].get_text()
            return horoscope
        else:
            return "No paragraphs found."
    else:
        return f"Failed to retrieve the webpage. Status code: {response.status_code}"

In [279]:
# define the function to put together the horoscope message that is to be sent out 
def get_horoscope_message(Sign):
    #call the previously created function
    horoscope = horoscope_strip(Sign)
    if "Invalid zodiac sign." in horoscope or "Failed to retrieve the webpage." in horoscope:
        return horoscope

    # Today's date
    today = datetime.now().date()
    today = today.strftime("%m/%d/%Y")

    # Create formatted horoscope message
    horoscope_message = f"Good Morning {Sign}!!!! The stars have spoken about your day!!! Today is {today}: {horoscope}"
    
    return horoscope_message


In [281]:
#make sure it works here 
#h=get_horoscope_message('Capricorn')

In [282]:
#send_imessage(h, 'lob@gmail.com')

In [283]:
#define the function we will use to actually send this message out via text... can only text to emails and can only text from Macs 
def send_imessage(message, recipient):
    script = f'''
    tell application "Messages"
        set targetService to 1st service whose service type = iMessage
        set targetBuddy to buddy "{recipient}" of targetService
        send "{message}" to targetBuddy
    end tell
    '''

    os.system(f"osascript -e '{script}'")

In [284]:
# Sample DataFrame you will pass through the functions.... names arent really important but just helped me to keep track of who i had 
data = {
    'Name': ['Bob','rob','mob','lob','fob','nob','sob','wob','pob','gob', 'cob', 'dob','tob'],
    'Sign': ['Gemini','Taurus','Aquarius','Aquarius','Leo','Libra','Aries','Taurus','Aquarius','Gemini', 'Gemini', 'Aquarius', 'Libra'],
    'Email': ['bob@comcast.net','rob@gmail.com','mob@ymail.com','lob@aol.com','fob@gmail.com','nob@stenhard.net','sob@gmail.com','wob@icloud.com','pob@icloud.com','gob@gmail.com', 'cob@gmail.com', 'dob@gmail.com', 'tob@gmail.com']
}
df = pd.DataFrame(data)

In [285]:
# df

In [286]:
# Define the job function
def job():
    for index, row in df.iterrows():
        name = row['Name']
        sign = row['Sign']
        email = row['Email'] 

        # Get the horoscope message
        horoscope_message = get_horoscope_message(sign)
        #send the horoscope message 
        if "Invalid zodiac sign." not in horoscope_message and "Failed to retrieve the webpage." not in horoscope_message:
            # Send the message via iMessage
            send_imessage(horoscope_message, email)

In [287]:
# Schedule the message for a specific time
schedule_time = "06:00"  # Format: HH:MM in 24-hour format
schedule.every().day.at(schedule_time).do(job)

print(f"Scheduled message at {schedule_time}. Awaiting...")

Scheduled message at 06:00. Awaiting...


In [ ]:
# Run the scheduler, this code needs to remain running as long as you want this message to be sent out....
while True:
    schedule.run_pending()
    time.sleep(1)